In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Import necessary library
import matplotlib.pyplot as plt
import seaborn as sns

# https://www.rookieroad.com/football/101/special-teams/

In [ ]:
# Reading data tables
games = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/games.csv')
plays = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/plays.csv')
#players = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/players.csv')
#tracking2018 = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/tracking2018.csv')
#tracking2019 = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/tracking2019.csv')
#tracking2020 = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/tracking2020.csv')
scouting = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2022/PFFScoutingData.csv')

In [ ]:
# Choosing games related to last 3 SuperBowl Winners
games_p1 = games.set_index('homeTeamAbbr')
games_p2 = games.set_index('visitorTeamAbbr')
winners = ['TB','KC','NE']
g1 = games_p1.loc[winners]
g2 = games_p2.loc[winners]
g1 = (g1.reset_index())#.set_index('gameId')
g2 = (g2.reset_index())#.set_index('gameId')

g = pd.concat([g1,g2]).drop_duplicates()
g = g.set_index('gameId')
#g.index.is_unique

p1 = plays.set_index('gameId')
pw = (p1.loc[list(g.index)]).reset_index()

In [ ]:
df = (pw.loc[:,['possessionTeam','specialTeamsPlayType']]).set_index('possessionTeam')
#Extra Point, Field Goal, Kickoff or Punt


In [ ]:
df1 = pd.DataFrame([list((df.loc['TB',:]).value_counts().sort_index(ascending=True)),
                    list((df.loc['KC',:]).value_counts().sort_index(ascending=True)),
                    list((df.loc['NE',:]).value_counts().sort_index(ascending=True))
                    ],
                    index = ['TB','KC','NE'], 
                    columns = ['Extra Point', 'Field Goal', 'Kickoff', 'Punt'])


In [ ]:
### Checking SpecialTeam Play Type for Last 3 SuperBowl Winners
fig, axes = plt.subplots(2,1)
df1.plot.bar(ax = axes[0], alpha = 0.5)

# Shrink current axis by 20%
box = axes[0].get_position()
axes[0].set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
axes[0].legend(loc='center left', bbox_to_anchor=(1, 0.5))


### Plot stacked version of the distribution
df1.plot.barh(ax = axes[1], stacked = True, alpha = 0.5)
# Shrink current axis by 20%
box1 = axes[1].get_position()
axes[1].set_position([box1.x0, box1.y0, box1.width * 0.8, box1.height * 0.9])

# Put a legend to the right of the current axis
axes[1].legend(loc='center left', bbox_to_anchor=(1, 0.5))



In [ ]:
# Understand percentage of return yards over kick length for SuperBowl Winners in "Return" playResult
df2 = pw.loc[:,['gameId','playId','possessionTeam','specialTeamsPlayType','specialTeamsResult','kickLength','kickReturnYardage']]
df2 = df2[df2['specialTeamsResult'] == 'Return']
df2 = df2.set_index('gameId')
#df2.head()

In [ ]:
games_p = games.loc[:,['gameId','homeTeamAbbr','visitorTeamAbbr']]
games_p = games_p.set_index('gameId')
#games_p.head()

In [ ]:
df3 = (pd.merge(df2, games_p, how='left',left_index=True,right_index=True)).reset_index()
#df3.head()

In [ ]:
# Tampa Bay Buccaneers
df4 = df3[(df3['homeTeamAbbr']=='TB') | (df3['visitorTeamAbbr']=='TB')]
df4 = df4[df4['possessionTeam'] != 'TB']
df4['Ratio'] = df4['kickReturnYardage']*100/df4['kickLength']
#df4['Ratio']

# Kansas City Chiefs
df5 = df3[(df3['homeTeamAbbr']=='KC') | (df3['visitorTeamAbbr']=='KC')]
df5 = df5[df5['possessionTeam'] != 'KC']
df5['Ratio'] = df5['kickReturnYardage']*100/df5['kickLength']
#df4['Ratio']

# New Englad Patriots
df6 = df3[(df3['homeTeamAbbr']=='NE') | (df3['visitorTeamAbbr']=='NE')]
df6 = df6[df6['possessionTeam'] != 'NE']
df6['Ratio'] = df6['kickReturnYardage']*100/df6['kickLength']
#df4['Ratio']



In [ ]:
fig, axes = plt.subplots(3,1, sharex = True)

axes[0].set_title('Tampa Bay Buccaneers')
axes[0].set_xlim([0, 100])
sns.histplot(df4['Ratio'], bins=range(1, 110, 3), ax = axes[0])

axes[1].set_title('Kansas City Chiefs')
axes[1].set_xlim([0, 100])
sns.histplot(df5['Ratio'], bins=range(1, 110, 3), ax = axes[1])

axes[2].set_title('New Englad Patriots')
axes[2].set_xlim([0, 100])
sns.histplot(df6['Ratio'], bins=range(1, 110, 3), ax = axes[2])

In [ ]:
#### Checking correlation between return ratio and other variables
scouting_p = scouting.loc[:,['gameId','playId','operationTime','hangTime',
                             'kickType','kickDirectionIntended','kickDirectionActual',
                             'returnDirectionIntended','returnDirectionActual','kickoffReturnFormation']]
scouting_p = scouting_p.set_index(['gameId','playId'])

In [ ]:
# Tampa Bay Buccaneers
df4 = df4.set_index(['gameId','playId'])
df7 = (pd.merge(df4, scouting_p, how='left',left_index=True,right_index=True)).reset_index()
df4 = df4.reset_index()
df7 = df7[df7['hangTime'].notnull()]

# Kansas City Chiefs
df5 = df5.set_index(['gameId','playId'])
df8 = (pd.merge(df5, scouting_p, how='left',left_index=True,right_index=True)).reset_index()
df5 = df5.reset_index()
df8 = df8[df8['hangTime'].notnull()]

# New Englad Patriots
df6 = df6.set_index(['gameId','playId'])
df9 = (pd.merge(df6, scouting_p, how='left',left_index=True,right_index=True)).reset_index()
df6 = df6.reset_index()
df9 = df9[df9['hangTime'].notnull()]


In [ ]:
fig, axes = plt.subplots(1,1, sharex = True)
sns.regplot(x= 'hangTime', y= 'Ratio', data = df7)
sns.regplot(x= 'hangTime', y= 'Ratio', data = df8)
sns.regplot(x= 'hangTime', y= 'Ratio', data = df9)

In [ ]:
fig, axes = plt.subplots(3,1, sharex = True)#, sharey = True)
sns.regplot(x= 'hangTime', y= 'Ratio', data = df7, ax = axes[0])
sns.regplot(x= 'hangTime', y= 'Ratio', data = df8, ax = axes[1])
sns.regplot(x= 'hangTime', y= 'Ratio', data = df9, ax = axes[2])